# Python建模库介绍

在本章，将回顾pandas的一些特性，这些特性可能会在使用pandas进行模型训练和评分有用。本章介绍两个流行的建模工具包——statsmodels \(http://statsmodels.org \)和scikit-learn \(http://scikit-learn.org \)。由于这两个项目每个都大到可以单独成书，这里就不再尝试完整地介绍，而是像其他基于Python的数据科学、统计学和机器学习的书籍一样，直接进入到项目的在线文档。

## pandas与建模代码的结合

使用pandas用于数据载入和数据清洗，之后切换到模型库去建立模型是一个常见的模型开发工作流程。在机器学习中，特征工程是模型开发的重要部份之一。特征工程是指从原生数据集中提取可用于模型上下文的有效信息的数据转换过程或分析。本书中学习的数据聚合和GroupBy工具常用于特征工程中。

尽管一个“好”的特征工程的细节已经超出了本书的范围，但在本书中仍然会展示一些可以再利用pandas进行数据操作和建模之间无痛切换的方法。

pandas和其他分析库的结合点通常是Numpy数组。要将DataFrame转换为Numpy数组，使用.values属性：

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.DataFrame({'x0':[1,2,3,4,5],'x1':[0.01,-0.01,0.25,-4.1,0.],'y':[-1.5,0.,3.6,1.3,-2.]})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [3]:
data.columns

Index(['x0', 'x1', 'y'], dtype='object')

In [4]:
data.values

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

将数组再转化为DataFrame，可以传递一个含有列名的二维ndarray：

In [5]:
df2 = pd.DataFrame(data.values, columns=['one','two','three'])
df2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


>.values属性一般在数据是同构化的时候使用——例如，都是数字类型的时候。如果数据是异构化的，**结果是Python对象的ndarray：**

In [6]:
df3 = data.copy()
df3['strings'] = list('abcde')
df3

,x0,x1,y,strings
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


In [7]:
df3.values

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

对于某些模型，可能只想使用一部分列。这里推荐使用loc索引和values：

In [8]:
model_cols = ['x0','x1']
data.loc[:,model_cols].values

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

有些库对pandas有本地化支持，可以自动做一下工作：将数据从DataFrame转换到Numpy中，并将模型参数名称附于输出表的列或Series上。在其他情况下，将不得不手动进行“元数据管理”的操作。

在第12章中，学习了pandas的Categorical类型和pandas.get_dummies函数。假设在示例数据集中，有一个非数字类型的列：

In [9]:
data['category'] = pd.Categorical(list('abaab'),categories=['a','b'])
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


In [10]:
dummies = pd.get_dummies(data.category, prefix='category')
data_with_dummies = data.drop('category', axis=1,).join(dummies)
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


>data.drop()：根据某个指定的标签删除行或列

在使用虚拟变量你和特定的统计模型时是有一些细微区别的。比如，当拥有不止简单的数字来行列时，使用Patsy可以更简单、更少出错。

## 使用Patsy创建模型描述

Patsy\(http://patsy.readthedocs.io/ \)是一个用于描述统计模型（尤其是线性模型）的Python库。它使用一种小型基于字符串的“公式语法”，这种语法受到了R、S统计编程语言中公式语法的启发（但是并不是完全相同）。

Patsy能够很好地支持statsmodels中特定的线性模型，因此本小节将专注与他的主要特性。Patsy的公式是特殊字符串语法，如下：

    y ～ x0 + x1

语法 a+b 并不代表a加b，而是指为模型而创建的设计矩阵中的列名。patsy.dmatrices函数在数据集上（可以是一个DataFrame或数组的字典）接收一个公式字符串，并返回了为一个线性模型产生的设计矩阵：

In [11]:
data = pd.DataFrame({
    'x0':[1,2,3,4,5],
    'x1':[0.01,-0.01,0.25,-4.1,0.],
    'y':[-1.5,0.,3.6,1.3,-2.]})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [12]:
import patsy

In [13]:
y, X = patsy.dmatrices('y ~ x0 + x1', data)

可以看到：

In [14]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [15]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

这些Patsy的DesingMatrix实例是含有附加元数据的Numpy ndarray：

In [16]:
np.asarray(y)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [17]:
np.asarray(X)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

Intercept（截距），是线性模型，如普通最小二乘回归中的惯例。可以通过给模型添加名词列“+0”来不显示截距：

In [18]:
patsy.dmatrices('y ~ x0 + x1 + 0',data)[1]

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

Patsy对象可以直接传递一些算法，比如numpy.linalg.lstsq等，这些算法都会执行一个最小二乘回归：

In [19]:
coef, resid, _, _ = np.linalg.lstsq(X,y)
#resid：残差
#X：系数矩阵
#y：因变量

D:\Python\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


模型元数据保留在design_info属性中，因此可以将模型列名重新附加到拟合系数一伙的一个Series，例如：

In [20]:
coef

array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

In [21]:
coef = pd.Series(coef.squeeze(),index=X.design_info.column_names)
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

In [22]:
X.design_info

DesignInfo(['Intercept', 'x0', 'x1'],
           factor_infos={EvalFactor('x0'): FactorInfo(factor=EvalFactor('x0'),
                                    type='numerical',
                                    state=<factor state>,
                                    num_columns=1),
                         EvalFactor('x1'): FactorInfo(factor=EvalFactor('x1'),
                                    type='numerical',
                                    state=<factor state>,
                                    num_columns=1)},
           term_codings=OrderedDict([(Term([]),
                                      [SubtermInfo(factors=(),
                                                   contrast_matrices={},
                                                   num_columns=1)]),
                                     (Term([EvalFactor('x0')]),
                                      [SubtermInfo(factors=(EvalFactor('x0'),),
                                                   contrast_matrices={},
     

### Patsy公式中的数据转换

可以将Python代码混合到Patsy公式中，在执行公式时，Patsy库将尝试在封闭的作用域中寻找要使用的函数：

In [23]:
y, X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1) + 1)', data)
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

一些常用的变量变换包括标准化（均值为0，方差为1）和中心化（减去均值）。Patsy有可以实现这个功能的内置的函数：

In [24]:
y, X = patsy.dmatrices('y ~ standardize(x0) + center(x1)', data)
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

作为建模的一部分过程，可能会在一个数据集上拟合一个模型，之后基于另一个模型评价该模型。这部分数据可能是保留下的部分数据或者之后在观察的新数据。在应用像中心化或标准化这样的转换时，基于新数据用模型进行预测时要小心。这些转换会被称为有状态的转换，因为在形成新数据集时必须使用源数据集中的均值或标准差等统计值。

patsy.build_design_matrices函数可以使用原始样本内数据集中保存的信息将变换应用于新的样本外数据上：

In [25]:
new_data = pd.DataFrame({
    'x0':[6,7,8,9],
    'x1':[3.1,-0.5,0,2.3],
    'y':[1,2,3,4]
})
new_X = patsy.build_design_matrices([X.design_info],new_data)
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

因为加号“+”在Patsy公式的上下文中并不表示加法的意思，当想要对数据集中两列按列名相加时，必须将列名封装到特殊的I函数中：

In [26]:
y, X = patsy.dmatrices('y ~ I(x0 + x1)', data)
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

在patsy.builtins模块中Patsy还有其他一些内建转换。更多内容请参见官方在线文档。

### 分类数据与Patsy

有多种方式可以将非数字类型数据转换为可以用于模型的设计矩阵。一个完整的解决方案超出了本社的范围。

当在Patsy公式中使用非数字名词列时，它们将会被默认转换为虚拟变量。如果有截距，就会去掉一个虚拟变量以避免共线性：

In [27]:
data = pd.DataFrame({
    'key1':list('aabbabab'),
    'key2':[0,1]*3 + [0,0],
    'v1':list(range(1,9)),
    'v2':[-1,0,2.5,-0.5,4.0,-1.2,0.2,-1.7]
})
y, X = patsy.dmatrices('v2 ~ key1', data)
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

如果忽略了模型的截距，每个类别值的列将会被包含在模型的设计矩阵中：

In [28]:
y, X = patsy.dmatrices('v2 ~ key1 + 0', data)
X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

数字类型可以使用C函数解释为分类模型：

In [29]:
y, X = patsy.dmatrices('v2 ~ C(key2)', data)
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

当在模型中使用多个分类名词时，事情可能会变得恶更复杂，因为会出现包含形式为key1：key2的交互项，这个交互项可被用于方差分析（ANOVA）模型：

In [30]:
data['key2'] = data['key2'].map({0:'zero',1:'one'})
data

,key1,key2,v1,v2
0,a,zero,1,-1.0
1,a,one,2,0.0
2,b,zero,3,2.5
3,b,one,4,-0.5
4,a,zero,5,4.0
5,b,one,6,-1.2
6,a,zero,7,0.2
7,b,zero,8,-1.7


In [31]:
y, X = patsy.dmatrices('v2 ~ key1 + key2', data)
X

DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [32]:
y, X = patsy.dmatrices('v2 ~ key1 + key2 + key1:key2', data)
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

Patsy提供转换分类数据的其它方法，包括以特定顺序转换。请参阅线上文档。

## statsmodels介绍

statsmodels \( http://www.statsmodels.org \)是一个Python库，用于拟合多种统计模型，执行统计测试以及数据探索和可视化。statsmodels包含更多的“经典”频率苏俄拍统计方法，而贝叶斯方法和机器学习模型可在其他库中找到。

包含在statsmoddels中的一些模型：
+ 线性模型，广义线性模型和鲁棒线性模型
+ 线性混合效应模型
+ 方差分析（ANOVA）方法
+ 时间序列过程和状态空间模型
+ 广义的矩量法

### 评估线性模型

统计模型中有几种线性回归模型，从较基本的（例如，普通最小二乘）到更复杂的（例如，迭代重新加权的最小二乘）。

statsmodels中的线性模块有两个不同的主要接口：基于数组的和基于公式的。这些接口通过API模块导入来访问：

    import statsmodels.api as sm
    import statsmoedls.formula.api as smf
    
为了展示如何使用这些接口，下面将根据一些随机数生成线性模型：

    def dnorm(mean, variance, size=1):
        if isinstance(size, int):
            size = size
        return mean + np.sqrt(variance) * np.random.randn(*size)
    
    # 用于复现
    np.random.seed(12345)
    N = 100
    X = np.c_[dnorm(0, 0.4,size=N), dnorm(0, 0.6,size=N), dnorm(0, 0.2,size=N)]
    #np.c_[]：将切片对象转换为沿第二轴的连接
    eps = dnorm(o, 0.1, size=N)
    beta = [0.1,0.3,0.5]
    
    y = np.dot(X, beta) + eps
    
这里，写下了已知参数beta“真实”模型。在这种情况下，dnorm适用于生成具有特定均值和方差的正太分布数据的辅助函数。所以有：

In [33]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [34]:
def dnorm(mean, variance, size=1):
    if isinstance(size, int):
        size = size
    return mean + np.sqrt(variance) * np.random.randn(size)

In [35]:
np.random.seed(12345)
N = 100
X = np.c_[dnorm(0, 0.4,size=N), dnorm(0, 0.6,size=N), dnorm(0, 0.2,size=N)]
#np.c_[]：将切片对象转换为沿第二轴的连接
eps = dnorm(0, 0.1, size=N)
beta = [0.1,0.3,0.5]
#“真实”模型
y = np.dot(X, beta) + eps

In [36]:
X[:5]

array([[-0.12946849, -1.21275292,  0.50422488],
       [ 0.30291036, -0.43574176, -0.25417986],
       [-0.32852189, -0.02530153,  0.13835097],
       [-0.35147471, -0.71960511, -0.25821463],
       [ 1.2432688 , -0.37379916, -0.52262905]])

In [37]:
y[:5]

array([ 0.42786349, -0.67348041, -0.09087764, -0.48949442, -0.12894109])

线性模型通常与在Patsy中看到的截距项相匹配。sm.add_constant函数可以截距列添加到现有矩阵：

In [38]:
X_model = sm.add_constant(X)
X_model[:5]

array([[ 1.        , -0.12946849, -1.21275292,  0.50422488],
       [ 1.        ,  0.30291036, -0.43574176, -0.25417986],
       [ 1.        , -0.32852189, -0.02530153,  0.13835097],
       [ 1.        , -0.35147471, -0.71960511, -0.25821463],
       [ 1.        ,  1.2432688 , -0.37379916, -0.52262905]])

sm.OLS类可以拟合一个最小二乘线性回归：

In [39]:
model = sm.OLS(y, X)

模型的fit方法返回一个回归结果对象，该对象包含了估计的模型参数和其他的诊断：

In [40]:
results = model.fit()
results.params

array([0.17826108, 0.22303962, 0.50095093])

在results上调用summary方法可以打印出一个模型的诊断细节：

In [41]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.430
Model:                            OLS   Adj. R-squared:                  0.413
Method:                 Least Squares   F-statistic:                     24.42
Date:                Wed, 22 May 2019   Prob (F-statistic):           7.44e-12
Time:                        13:46:09   Log-Likelihood:                -34.305
No. Observations:                 100   AIC:                             74.61
Df Residuals:                      97   BIC:                             82.42
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.1783      0.053      3.364      0.0

此处参数的名称已被赋予通用名称x1、x2等。假设所有模型参数都在DataFrame中：

In [42]:
data = pd.DataFrame(X, columns=['col0', 'col1', 'col2'])
data['y'] = y
data[:5]

,col0,col1,col2,y
0,-0.129468,-1.212753,0.504225,0.427863
1,0.302910,-0.435742,-0.254180,-0.673480
2,-0.328522,-0.025302,0.138351,-0.090878
3,-0.351475,-0.719605,-0.258215,-0.489494
4,1.243269,-0.373799,-0.522629,-0.128941


现在可以使用statsmodels公式API和Patsy公式字符串：

In [43]:
results = smf.ols('y ~ col0 + col1 + col2', data=data).fit()
results.params

Intercept    0.033559
col0         0.176149
col1         0.224826
col2         0.514808
dtype: float64

In [44]:
results.tvalues

Intercept    0.952188
col0         3.319754
col1         4.850730
col2         6.303971
dtype: float64

观察statsmodels如何将结果作为带有DataFrame列名称的Series返回。使用公式和pandas对象时，也不需要使用add_constant。

给定新的样本外数据后，可以根据估计的模型参数计算预测值：

In [45]:
results.predict(data[:5])

0   -0.002327
1   -0.141904
2    0.041226
3   -0.323070
4   -0.100535
dtype: float64

还有很多额外的工具，可针对在statsmodels中能够探索的线性模型结果进行分析、诊断和可视化。除了普通最小二乘法之外，还有其他种类的线性模型。

### 评估时间序列处理

statsmodels中的另一类模型用于时间序列分析。其中包括自回归过程，卡尔曼滤波和其他状态空间模型，以及多变量自回归模型。

接下来，模拟一些具有自回归结构和噪声的时间序列数据：

    init_x = 4
    
    import random
    values = [init_x, init_x]
    N = 1000
    
    b0 = 0.8
    b1 = -0.4
    noise = dnorm(0, 0.1, N)
    for i in range(N):
        new_x = values[-1] * b0 + values[-2] * b1 + noise[i]
        values.append(new_x)
        
该数据具有参数为0.8和-0.4的AR（2）结构（两个滞后）。当拟合一个AR模型时，可能不知道包含的滞后项的数量，所以可以用更大的滞后数量来拟合模型：

In [46]:
import random

In [47]:
init_x = 4

values = [init_x, init_x]
N = 1000

b0 = 0.8
b1 = -0.4
noise = dnorm(0, 0.1, N)
for i in range(N):
    new_x = values[-1] * b0 + values[-2] * b1 + noise[i]
    values.append(new_x)

In [48]:
MAXLAGS = 5
model = sm.tsa.AR(values)
results = model.fit(MAXLAGS)

结果中的估计参数首先是截距，接下来是前两个滞后的估计：

In [49]:
results.params

array([-0.00616093,  0.78446347, -0.40847891, -0.01364148,  0.01496872,
        0.01429462])

这些模型的深层细节以及如何解释其结果超出了本书所能涵盖的范围，但在statsmodels文档中还有很多可以发现的内容。

## scikit-learn介绍

scikit-learn\( http://scikit-learn.org \)是使用最广泛且最受信任的通用Python机器学习库。它包含广泛的标准监督的和无监督的机器学习方法，包括用于模型选择和评估、数据转换、数据加载和模型持久化的工具。这些模型可用于分类、聚类、预测和其他常见任务。

有很多优秀的在线和印刷资源可用于学习机器学习，以及如何应用scikit-learn和TensorFlow等库来解决实际问题。在本节中，将简要介绍一下scikit-learn API风格。

下面，使用Kaggle\( https://www.kaggle.com/c/titanic \)中关于泰坦尼克好上生还乘客的经典数据集，其中泰坦尼克好于1912年沉没。使用pandas载入测试和训练数据集：

In [50]:
train = pd.read_csv('C:/Users/lenovo/data_of_python/examples/titanic/train.csv')
test = pd.read_csv('C:/Users/lenovo/data_of_python/examples/titanic/test.csv')
train[:4]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


像statsmodels和scikit-learn通常不能接收缺失数据，因此要检查各列，看看是否有包含缺失数据：

In [51]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [52]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

像这样的统计和机器学习的例子中，一个典型的任务是根据数据中的特征来预测乘客是否能幸存下来。将模型拟合到训练数据集上，然后在样本外测试数据集上进行评估。

如果想用Age作为预测，但缺少数据。有很多方法可以进行缺失数据插补（imputation），这里只做一个简单的插补，并使用训练数据集的中位数填充两个表中的空值：

In [53]:
impute_value = train['Age'].median()
train['Age'] = train['Age'].fillna(impute_value)
test['Age'] = test['Age'].fillna(impute_value)

现在要确定模型，并且添加一列IsFemale作为‘SEX’列的编码版：

In [54]:
train['IsFemale'] = (train['Sex'] == 'female').astype(int)
test['IsFemale'] = (test['Sex'] == 'female').astype(int)

然后决定一些模型变量并创建NumPy数组：

In [55]:
predictors = ['Pclass', 'IsFemale', 'Age']
X_train = train[predictors].values
X_test = test[predictors].values
y_train = train['Survived'].values
X_train[:5]

array([[ 3.,  0., 22.],
       [ 1.,  1., 38.],
       [ 3.,  1., 26.],
       [ 1.,  1., 35.],
       [ 3.,  0., 35.]])

不能保证这是一个好的模型，同时也不能保证这些特征都符合。这里使用scikit-learn的LogisticRegression模型创建一个模型实例：

In [56]:
from sklearn.linear_model import LogisticRegression

In [57]:
model = LogisticRegression()

与statsmodels类似，可以使用模型的fit方法在训练数据上拟合模拟：

In [58]:
model.fit(X_train, y_train)

D:\Python\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

使用model.predict可以为测试数据集形成预测

In [59]:
y_predict = model.predict(X_test)
y_predict[:10]

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0], dtype=int64)

如果拥有测试数据集的真实值，则可以计算精度百分比或其他一些错误指标：

    (y_true == y_predict).mean()
    
实际上，模型训练中经常存在许多附加的复杂层次。许多模型具有可以调整的参数，并且存在可用于参数调整的交叉验证等技术避免过度拟合训练数据。这通常可以在新数据上产生更好的预测性能或稳健性。

交叉验证通过分割训练数据来模拟样本外预测。基于像均方误差值之类的模型准确度分数，可以对模型参数执行网络搜索。一些模型，如逻辑回归，具有内置交叉验证的估计类。例如，LogisticRegressionCV类可以与一个参数一起使用，该参数表示网格搜索在模型正则化参数C上的细致度：

In [60]:
from sklearn.linear_model import LogisticRegressionCV

In [61]:
model_cv = LogisticRegressionCV(10)
model_cv.fit(X_train,y_train)

D:\Python\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


LogisticRegressionCV(Cs=10, class_weight=None, cv='warn', dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='warn', n_jobs=None, penalty='l2',
           random_state=None, refit=True, scoring=None, solver='lbfgs',
           tol=0.0001, verbose=0)

要手动进行交叉验证，可以使用cross_val_score帮助函数，该函数处理数据拆分过程。例如，为了用模型与训练数据的四个非重叠分割进行交叉验证，可以这样做：

In [62]:
from sklearn.model_selection import cross_val_score

In [63]:
model = LogisticRegression(C=10)
scores = cross_val_score(model, X_train, y_train, cv=4)
scores

D:\Python\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Python\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Python\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Python\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


array([0.77232143, 0.80269058, 0.77027027, 0.78828829])

默认评分指标是依赖于模型的，但可以指定明确的评分函数。经过交叉验证的模型需要更长时间的训练，但通常可以产生更好的模型性能。

我只是介绍了一些Python建模库的表面内容，现在有越来越多的框架用于各种统计和机器学习，它们都是用Python或Python用户界面实现的。
这本书的重点是数据规整，有其它的书是关注建模和数据科学工具的。其中优秀的有：

+ Andreas Mueller and Sarah Guido (O’Reilly)的 《Introduction to Machine Learning with Python》
+ Jake VanderPlas (O’Reilly)的 《Python Data Science Handbook》
+ Joel Grus (O’Reilly) 的 《Data Science from Scratch: First Principles》
+ Sebastian Raschka (Packt Publishing) 的《Python Machine Learning》
+ Aurélien Géron (O’Reilly) 的《Hands-On Machine Learning with Scikit-Learn and TensorFlow》

虽然书是学习的好资源，但是随着底层开源软件的发展，书的内容会过时。最好是不断熟悉各种统计和机器学习框架的文档，学习最新的功能和API。
